In [32]:
import pandas as pd
import numpy as np
import re

In [ ]:
dicom_metadata = pd.read_csv('../data/UCSF_ComeBACK/xnat.radiology.ucsf.edu/dicom_metadata.csv')

/var/folders/8l/dkxhdycd15d4pt_h700hp40h0000gq/T/ipykernel_42210/2963419399.py:1: DtypeWarning: Columns (11,16,62,65,67,77,79,82,102,103,104,105,126,132) have mixed types. Specify dtype option on import or set low_memory=False.
  dicom_metadata = pd.read_csv('/Users/tmcsween21/Documents/data/UCSF_ComeBACK/xnat.radiology.ucsf.edu/dicom_metadata.csv')


In [ ]:
dicom_metadata = dicom_metadata[~dicom_metadata['FileName'].str.contains('.DS_Store')]
dicom_metadata = dicom_metadata[~dicom_metadata['FileName'].str.contains('upload.unsupported')]

In [ ]:
dicom_metadata_grouped = dicom_metadata.groupby('PathToFolder')['InstanceNumber'].apply(list).reset_index()

In [ ]:
dicom_metadata_grouped['num_slices'] = dicom_metadata_grouped['InstanceNumber'].apply(lambda x: np.floor(len(x)/3))

In [ ]:
dicom_metadata_grouped['InstanceNumber'] = dicom_metadata_grouped['InstanceNumber'].apply(lambda x: sorted(x))

In [ ]:
dicom_metadata_grouped['InstanceNumberSelected'] = dicom_metadata_grouped.apply(lambda x: x['InstanceNumber'][int(x['num_slices']):], axis=1)
dicom_metadata_grouped['InstanceNumberSelected'] = dicom_metadata_grouped.apply(lambda x: x['InstanceNumberSelected'][:-int(x['num_slices'])], axis=1)

In [ ]:
dicom_metadata_grouped = dicom_metadata_grouped.drop(columns=['InstanceNumber', 'num_slices'])
dicom_metadata_grouped = dicom_metadata_grouped.rename(columns={'InstanceNumberSelected': 'InstanceNumber'})

In [ ]:
dicom_metadata_grouped = dicom_metadata_grouped.explode('InstanceNumber')

In [ ]:
dicom_metadata = dicom_metadata.merge(dicom_metadata_grouped, on=['InstanceNumber', 'PathToFolder'])

In [ ]:
dicom_metadata.to_csv('../data/UCSF_ComeBACK/xnat.radiology.ucsf.edu/dicom_metadata_mid_third_slices.csv', index=False)

In [ ]:
dicom_metadata['duplicates'] = dicom_metadata.duplicated(subset=['FileName'], keep=False)
dicom_metadata['duplicates'] = dicom_metadata['duplicates'].astype(int)
duplicates = dicom_metadata['duplicates'].sum()
print(f'There are {duplicates} duplicates in the dataframe')

There are 0 duplicates in the dataframe


In [ ]:
control_metadata = pd.read_csv('../data/UCSF_ComeBACK/COMEBACK_Controls/data/control_metadata.csv')

In [ ]:
control_metadata = control_metadata[control_metadata['FileName'].str.contains('_T2w_')]

In [ ]:
control_metadata_grouped = control_metadata.groupby('PathToFolder')['InstanceNumber'].apply(list).reset_index()

In [ ]:
control_metadata_grouped['num_slices'] = control_metadata_grouped['InstanceNumber'].apply(lambda x: np.floor(len(x)/3))
control_metadata_grouped['InstanceNumber'] = control_metadata_grouped['InstanceNumber'].apply(lambda x: sorted(x))
control_metadata_grouped['InstanceNumberSelected'] = control_metadata_grouped.apply(lambda x: x['InstanceNumber'][int(x['num_slices']):], axis=1)
control_metadata_grouped['InstanceNumberSelected'] = control_metadata_grouped.apply(lambda x: x['InstanceNumberSelected'][:-int(x['num_slices'])], axis=1)
control_metadata_grouped = control_metadata_grouped.drop(columns=['InstanceNumber', 'num_slices'])
control_metadata_grouped = control_metadata_grouped.rename(columns={'InstanceNumberSelected': 'InstanceNumber'})
control_metadata_grouped = control_metadata_grouped.explode('InstanceNumber')
control_metadata = control_metadata.merge(control_metadata_grouped, on=['InstanceNumber', 'PathToFolder'])

In [ ]:
import os
import pydicom
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

def dicom_to_png(dicom_filepath, output_folder, instance_number):
    """
    Converts a DICOM file to PNG format and saves it in the specified output folder.
    
    Args:
    - dicom_filepath (str): Path to the DICOM file.
    - output_folder (str): Path to the folder where the PNG will be saved.
    
    Returns:
    - str: Path to the saved PNG file.
    """
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_filepath)
    
    # Extract the pixel array and convert it to a numpy array
    image_raw = dicom_data.pixel_array.astype(np.float32)
    image_raw = np.array(image_raw)
    
    image_raw = Image.fromarray(image_raw)
    
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    participant_id = dicom_filepath.replace('../data/UCSF_ComeBACK/xnat.radiology.ucsf.edu/San_Francisco_cohort/', '')
    participant_id = participant_id.replace('/', '_')
    participant_id = participant_id.replace('.dcm', '')
    participant_id = participant_id.replace('.DCM', '')
    participant_id = participant_id + '_' + str(instance_number) + '.png'

    output_filepath = os.path.join(output_folder, participant_id)
    
    # Save the image as a PNG file
    # cv2.imwrite(output_filepath, image_raw)
    plt.imsave(output_filepath, image_raw, cmap='gray')
    
    return print("pngs saved")

In [ ]:
dicom_metadata_SF_cohort = dicom_metadata[dicom_metadata['PathToFolder'].str.contains('CMBK-0101')]
dicom_metadata_other_sites = dicom_metadata[~dicom_metadata['PathToFolder'].str.contains('CMBK-0101')]
dicom_metadata_sample = dicom_metadata_SF_cohort.sample(50)

dicom_metadata_sample['PathToFolder'] = '../data/UCSF_ComeBACK/xnat.radiology.ucsf.edu/' + dicom_metadata_sample['PathToFolder']
dicom_metadata_sample['dicom_filepath'] = dicom_metadata_sample.apply(lambda x: os.path.join(x['PathToFolder'], x['FileName']), axis=1)

for i in dicom_metadata_sample['dicom_filepath']:
    instance_number = dicom_metadata_sample[dicom_metadata_sample['dicom_filepath'] == i]['InstanceNumber'].values[0]
    # make the instance number an integer
    instance_number = int(instance_number)
    dicom_to_png(i, '../data/UCSF_ComeBACK/xnat.radiology.ucsf.edu/San_Francisco_cohort_pngs', instance_number)

pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
pngs saved
